### Human input

In [ ]:
import os
from dotenv import load_dotenv
from typing import Annotated


from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.types import Command, interrupt

from langsmith import Client

# Load environment variables from .env file
load_dotenv()

# Ensure the required API keys are set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set your OPENAI_API_KEY in the .env file")
if not os.getenv("TAVILY_API_KEY"):
    raise ValueError("Please set your TAVILY_API_KEY in the .env file")
if not os.getenv("LANGSMITH_API_KEY"):
    raise ValueError("Please set your LANGSMITH_API_KEY in the .env file")


class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize the state graph and the language model.
graph_builder = StateGraph(State)

# highlight-next-line
@tool
# highlight-next-line
def human_assistance(query: str) -> str:
    # highlight-next-line
    """Request assistance from a human."""
    # highlight-next-line
    human_response = interrupt({"query": query})
    # highlight-next-line
    return human_response["data"]


tavily_search = TavilySearchResults(max_results=2)
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [tavily_search, human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # Because we will be interrupting during tool execution,
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume.
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step

graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
# Compile the graph with a MemorySaver for persistence.
checkpointer = MemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
user_input = "I need some expert guidance for building an AI agent. Could you request assistance for me?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

Let's take a closer look at the `human_assistance` tool:

```python
@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]
```

Similar to Python's built-in `input()` function, calling `interrupt` inside the tool will pause execution. Progress is persisted based on our choice of [checkpointer](../../concepts/persistence/#checkpointer-libraries)-- so if we are persisting with Postgres, we can resume at any time as long as the database is alive. Here we are persisting with the in-memory checkpointer, so we can resume any time as long as our Python kernel is running.

To resume execution, we pass a [Command](../../concepts/human_in_the_loop/#the-command-primitive) object containing data expected by the tool. The format of this data can be customized based on our needs. Here, we just need a dict with a key `"data"`:

In [ ]:
human_response = (
    "We, the experts are here to help! We'd recommend you check out LangGraph to build your agent."
    " It's much more reliable and extensible than simple autonomous agents."
)

human_command = Command(resume={"data": human_response})

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

### Customizing State

In [1]:
import os
from dotenv import load_dotenv
from typing import Annotated


from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.types import Command, interrupt

from langsmith import Client

# Load environment variables from .env file
load_dotenv()

# Ensure the required API keys are set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Please set your OPENAI_API_KEY in the .env file")
if not os.getenv("TAVILY_API_KEY"):
    raise ValueError("Please set your TAVILY_API_KEY in the .env file")
if not os.getenv("LANGSMITH_API_KEY"):
    raise ValueError("Please set your LANGSMITH_API_KEY in the .env file")

class State(TypedDict):
    messages: Annotated[list, add_messages]
    # highlight-next-line
    name: str
    # highlight-next-line
    birthday: str



@tool
# Note that because we are generating a ToolMessage for a state update, we
# generally require the ID of the corresponding tool call. We can use
# LangChain's InjectedToolCallId to signal that this argument should not
# be revealed to the model in the tool's schema.
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """Request assistance from a human."""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        },
    )
    # If the information is correct, update the state as-is.
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # Otherwise, receive information from the human reviewer.
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # This time we explicitly update the state with a ToolMessage inside
    # the tool.
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # We return a Command object in the tool to update our state.
    return Command(update=state_update)


tavily_search = TavilySearchResults(max_results=2)
llm = ChatOpenAI(model="gpt-4o-mini")
tools = [tavily_search, human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # Because we will be interrupting during tool execution,
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume.
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

# Initialize the state graph and the language model.
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Compile the graph with a MemorySaver for persistence.
checkpointer = MemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer)

In [5]:
user_input = (
    "Can you look up when LangFuse was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangFuse was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_DfgXWJI6BUfXQiuqBXBBs3xa)
 Call ID: call_DfgXWJI6BUfXQiuqBXBBs3xa
  Args:
    query: LangFuse release date
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://langfuse.com/blog/launch-week-1", "content": "We're excited to announce Langfuse's first launch week. We're kicking it off on Monday April 22nd and will release a major upgrade to the Langfuse platform"}, {"url": "https://www.producthunt.com/products/langfuse/launches", "content": "Langfuse Prompt Experiments Open Source LLM Engineering Platform. Launched on November 22nd, 2024 ; Langfuse 2.0 the open source LLM engineering platform."

In [6]:
human_command = Command(
    resume={
        "name": "LangFuse",
        "correct": "yes"
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_5reyZ23BOLa6KVbaVmY172sJ)
 Call ID: call_5reyZ23BOLa6KVbaVmY172sJ
  Args:
    name: LangFuse
    birthday: 2024-11-22
================================= Tool Message =================================
Name: human_assistance

Correct
================================== Ai Message ==================================

LangFuse was released on November 22, 2024.


In [ ]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [4]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph', 'birthday': '2025-02-06'}

In [ ]:
graph.update_state(config, {"name": "LangGraph (library)"})

In [ ]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}